## Import Required Packages

In [1]:
import os
import shutil
from collections import Counter
import numpy as np
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, ElectraForQuestionAnswering, DataCollatorWithPadding,BertModel, ElectraForSequenceClassification, ElectraModel
from Preprocess.arabertpreprocess import ArabertPreprocessor
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import random
random.seed(1)
torch.manual_seed(3407)

# Load models

In [2]:
def load_models():
    def load_ckp(checkpoint_fpath, model, optimizer):
        """
        checkpoint_path: path to saved checkpoint
        model: model to load checkpoint parameters into       
        optimizer: optimizer defined in previous training
        """
        # load check point
        checkpoint = torch.load(checkpoint_fpath)
        # initialize state_dict from checkpoint to model
        model.load_state_dict(checkpoint['state_dict'])
        # initialize optimizer from checkpoint to optimizer
        optimizer.load_state_dict(checkpoint['optimizer'])
        # initialize valid_loss_min from checkpoint to valid_loss_min
        results = checkpoint['result_dict']
        # return model, optimizer, epoch value, min validation loss 
        return model, optimizer, checkpoint['epoch'], results
    model_name = model_name = "aubmindlab/araelectra-base-discriminator"
    araelectra_tokenizer = AutoTokenizer.from_pretrained(model_name,do_lower_case=False) 
    Cls_AraElectra = ElectraForSequenceClassification.from_pretrained(model_name, num_labels=2)
    QA_AraElectra = ElectraForQuestionAnswering.from_pretrained(model_name)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    learning_rate = 3e-5
    optimizer = torch.optim.Adam(Cls_AraElectra.parameters(), lr=learning_rate)
    cls_criterion = nn.CrossEntropyLoss()
    Cls_AraElectra.to(device)
    cls_model, optimizer, start_epoch, result_dict = load_ckp('Runs/AraElectraDecoupled/train/cls/first/best.pt', Cls_AraElectra, optimizer)
    span_learning_rate = 3e-5
    span_optimizer = torch.optim.AdamW(QA_AraElectra.parameters(), lr=span_learning_rate)
    QA_AraElectra.to(device)
    span_model, optimizer, start_epoch, result_dict = load_ckp('Runs/AraElectraDecoupled/train/span/first/best.pt', QA_AraElectra, optimizer)
    return span_model, cls_model
span_model, cls_model = load_models()

Some weights of the model checkpoint at aubmindlab/araelectra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at aubmindlab/araelectra-base-discriminator and are newly initialized: 

# Print Sample 

In [8]:
def get_sample():
    def add_end_index(answer, context):
        ## 1 if span match the context 0 otherwise
        text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(text)
        answer['answer_end'] = end_idx
        if text == context[start_idx:end_idx]:
            answer['answer_end'] = end_idx
            return False
        for i in range(1, 3):
            if text == context[start_idx - i:end_idx - i]:
                answer['answer_end'] = end_idx - 1
                answer['answer_start'] = start_idx - 1
                return False
        return True

    def arabert_preprocess(context, question, answer, arabert_prep):
        answer['text'] = arabert_prep.preprocess(answer['text'])
        context = arabert_prep.preprocess(context)
        question = arabert_prep.preprocess(question)
        res = context.find(answer['text'])
        if res != -1:
            answer['answer_start'] = res
        return context, question, answer, res

    def Read_AAQAD(path, arabert_prep):
        contexts = []
        answers = []
        questions = []
        IDs = []
        plausible = []
        cnt = 0
        with open(path) as f:
            aaqad_dict = json.load(f)
            for article in aaqad_dict['data']:
                for passage in article['paragraphs']:
                    context = passage['context']
                    for qa in passage['qas']:
                        question = qa['question']
                        if 'plausible_answers' in qa.keys():  # there is two cases if the question have no answer then use plausible answer
                            access = 'plausible_answers'
                            plausible.append(True)
                        else:
                            access = 'answers'
                            plausible.append(False)
                        for answer in qa[access]:
                            context, question, answer, res = arabert_preprocess(context, question, answer, arabert_prep)
                            # if res==-1:
                            #  cnt+=1
                            #  continue
                            flag = add_end_index(answer, context)  # if false dont add the
                            cnt = cnt + flag
                            flag = False
                            if not flag:
                                contexts.append(context)
                                answers.append(answer)
                                questions.append(question)
                                IDs.append(int(qa['id']))
        return contexts, questions, answers, plausible, IDs

    model_name = "araelectra-base-discriminator"
    arabert_prep = ArabertPreprocessor(model_name=model_name)
    aqad_test_contexts, aqad_test_questions, aqad_test_answers, aqad_test_plausible, aqad_test_ids = Read_AAQAD(
        'Data/AAQAD-test.json', arabert_prep)
    random.seed(3)
    rand_idx = [random.randint(0, len(aqad_test_contexts)) for _ in range(2)]
    rand_idx

    # [974, 2427]
    # plausible answer true means isimpossible True
    def get_info(rand_idx):
        contexts, questions, answers, plausible, IDs = [], [], [], [], []
        for i, idx in enumerate(rand_idx):
            print(
                f'Sample {i + 1} \nContext: {aqad_test_contexts[idx]} \nQuestion: {aqad_test_questions[idx]} \nIs Impossible to Answer? {aqad_test_plausible[idx]}')
            print("---------------------------")
            contexts.append(aqad_test_contexts[idx])
            questions.append(aqad_test_questions[idx])
            answers.append(aqad_test_answers[idx])
            plausible.append(aqad_test_plausible[idx])

        return contexts, questions, answers, plausible, IDs

    demo_contexts, demo_questions, demo_answers, demo_plausible, demo_IDs = get_info(rand_idx)
    return demo_contexts, demo_questions 
#demo_contexts, demo_questions = get_sample()


# Get Predictions

In [9]:
def get_predictions(demo_contexts, demo_questions, cls_model, span_model):
    #Creating the tokenizer
    model_name = model_name = "aubmindlab/araelectra-base-discriminator"
    araelectra_tokenizer = AutoTokenizer.from_pretrained(model_name,do_lower_case=False)
    demo_encodings = araelectra_tokenizer(demo_contexts, demo_questions, truncation=True, padding = True, return_tensors= 'pt')
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    cls_model.eval()
    span_model.eval()
    soft = torch.nn.Softmax(dim=1)
    tokens = demo_encodings['input_ids'].to(device)
    masks = demo_encodings['attention_mask'].to(device)
    tokens_type = demo_encodings['token_type_ids'].to(device)
    outputs = span_model(tokens, masks, tokens_type)
    no_probs = cls_model(tokens, masks, tokens_type)
    no_probs = soft(no_probs.logits)
    curr_batch_size = tokens.shape[0]
      #print(curr_batch_size)
      #print(outputs.start_logits.shape)
      #print(no_probs.logits.shape)
    for i in range(curr_batch_size):
        #print(f"this is tensor index {i}")
        start_pred, end_pred= torch.argmax(no_probs[i], dim=0), torch.argmax(no_probs[i], dim = 0)
        #print(start_pred.shape, end_pred.shape)
        #print(start_pred)
        #print(start_pred, end_pred)
        text_prediction = araelectra_tokenizer.decode(tokens[i][start_pred.item():end_pred.item()], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        probs_prediction = no_probs[i][1].item()
        if probs_prediction>=0.5:
            print(f'Sample {i+1} Prediction: \nAnswer: Can not be answered with Confidence: {probs_prediction}')
            print("---------------------------")

        else:
            print(f'Sample {i+1} Prediction \nAnswer: {text_prediction} with Confidence: {probs_prediction} with Start: {start_pred.item()} and End: {end_pred.item()}')
            print("---------------------------")


In [10]:
demo_contexts, demo_questions = get_sample()
get_predictions(demo_contexts, demo_questions, cls_model, span_model)

Sample 1 
Context: نقص الزنك له عواقب وخيمة على نمو الأطفال ، بحيث يؤدي إلى نقص في الطول و الإصابة بالتأخر النمو البنيوي ، فقد أظهرت دراسات للتدخلات التي أجريت في عدة بلدان وجود علاقة إيجابية بين توفير جرع الزنك المكملة والنمو الخطي لدى الأطفال . ويسبب نقصه أيضا مرض الإسهال و التهابات الجهاز التنفسي . 
Question: لماذا لا ينبغي أن يعطى الزنك بمفرده لأولئك الذين ليس لديهم أوجه قصور ؟ 
Is Impossible to Answer? True
---------------------------
Sample 2 
Context: في عام 1045 ، غزت الإمبراطورية البيزنطية أرمينيا . وبعدها بفترة قصيرة تلتها الدويلات الأخرى تحت السيطرة البيزنطية . لم يدم الحكم البيزنطي طويلا ، حيث انتصر السلاجقة في 1071 على البيزنطيين وسيطروا على أرمينيا في معركة ملاذكرد وأقاموا الإمبراطورية السلجوقية . هربا من الموت أو العبودية على يد أولئك الذين اغتالوا غاجيك الثاني ملك آني ، فر أحد أقاربه وهو أرمني يدعى روبن مع بعض أهل القرى إلى المضائق الجبلية في جبال طوروس ومن ثم إلى طرسوس في قيليقية . وفر لهم الحاكم البيزنطي مأوى في قصره حيث تأسست في نهاية المطاف مملكة قيليقية الأرمنية . 